# Tarea No.4: Árboles de desición

### Equipo No. 7:

**Aide Jazmín González Cruz**

**Elena Villalobos Nolasco**

**Yusuri Arciga Reyes**

El objetivo de la última tarea consistirá en modificar el Jupyter notebook que se proporcionó con el código CaRT para que además de resolver problemas de clasificación sirva para resolver problemas de regresión. Probarlo con el set de datos de bateadores que se utilizó en clase para que puedan validar que está funcionando bien. Compararlo contra el resultado en R. Para ello habrá que importar el data frame de R en python y convertirlo en una lista de listas.

In [1]:
# Importando librerias necesarias
import pandas as pd
import numpy as np
import statistics

## Cargando datos

In [2]:
hitters = pd.read_csv('Hitters.csv')
hitters.shape

(263, 21)

In [3]:
hitters.columns

Index(['Unnamed: 0', 'AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years',
       'CAtBat', 'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'League',
       'Division', 'PutOuts', 'Assists', 'Errors', 'Salary', 'NewLeague'],
      dtype='object')

In [4]:
# Delete first column (names)
hitters.drop(hitters.columns[[0]], axis=1, inplace=True)
hitters.head(5)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,6.163315,N
1,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,6.173786,A
2,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,6.214608,N
3,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,4.516339,N
4,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,6.620073,A


Si se usan todas las variables incluyendo las categóricas, estás serían algunos inputs en las funciones

In [5]:
num_col_salary = 18
# Column labels.
# These are used only to print the tree.
header = ["AtBat","Hits","HmRun","Runs","RBI","Walks","Years",    
          "CAtBat","CHits","CHmRun","CRuns","CRBI","CWalks","League" ,  
          "Division","PutOuts","Assists","Errors","Salary","NewLeague",
          "salary_c"]

In [6]:
hitters["Salary"].mean()

5.9272215412213916

In [7]:
# DEjando sólo las variable númericas
hitters.drop(['League'], axis=1, inplace=True)
hitters.drop(['Division'], axis=1, inplace=True)
hitters.drop(['NewLeague'], axis=1, inplace=True)
hitters.head(5)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary
0,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,6.163315
1,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,6.173786
2,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,6.214608
3,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,4.516339
4,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25,6.620073


In [8]:
quitar = ["AtBat","HmRun","Runs","RBI","Walks",  
          "CAtBat","CHits","CHmRun","CRuns","CRBI","CWalks",
          "PutOuts","Assists","Errors"]
hits_years = hitters.copy()
hits_years = pd.DataFrame(hits_years.drop(quitar, axis=1))
hits_years.head(5)

,Hits,Years,Salary
0,81,14,6.163315
1,130,3,6.173786
2,141,11,6.214608
3,87,2,4.516339
4,169,11,6.620073


### Pasando a Lista de Listas

- Para todas las variable númericas

In [9]:
hitters_lol = hitters.values.tolist()

- Sólo usando `hits` y `years`

In [10]:
hits_years_lol = hits_years.values.tolist()

### Funciones profesor

In [11]:
# Funciones profesor
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])
    #set regresa un conjunto({}) y regresa valores únicos

# Para la regresión mayor a menor y los promedios de datos subsecuentes y vecinos
# pe entre 2 y 3 -> 2.5 

def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[np.round(label,1)] = 0
        counts[np.round(label,1)] += 1
    return counts

def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

class Question:
    """A Question is used to partition a dataset.
    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))
    
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

# Para clasificación
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.
    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

# Caída del Gini
def info_gain(left, right, current_uncertainty):
    """Information Gain.
    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [12]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [13]:
# Clase hoja, guarda los valores de las etiquetas
# En el caso de regresión aquí van los promedios o determinar promedios
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [14]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [15]:
# Cosntuye árbol, termina cuando la ganancia es 0, y ya no se pudo subdividir y se 
# crea una hoja, si no es cero genera 2 subconjuntos para crear 2 ramas
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [16]:
# Impresión del árbol
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict ", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

## Funciones realizadas para variables continúas

In [17]:
# Funciones adicionales creadas
#Creando promedios entre valores
def promedio(lista):
    #print(lista)
    prom = []
    for i in range(len(lista)-1):
        prom.append((lista[i]+lista[i+1])/2)
    return prom

# Extrae una columna de una lista de listas dada
def Extract(lst, col_num): 
    return [item[col_num] for item in lst] 

In [18]:
# Para regresión (Error error cuadrado medio para cortes en tarea)
def rss(rows, salary_col):
    """
    Calcula la suma de residuos al cuadrado (RSS)
    """
    _rss = 0
    col = Extract(rows, salary_col)
    mean = statistics.mean(col)
    for i in range(len(col)-1):
        _rss = _rss + (col[i] - mean)**2
        
    return _rss

In [19]:
r = rss(hitters_lol,16)
r

206.19228672549767

In [20]:
# Entropía
def info_gain2(left, right, current_uncertainty, salary_col):
    """Information Gain.
    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * rss(left, salary_col)-\
                                (1 - p) * rss(right, salary_col)

In [21]:
def find_best_split2(rows, salary_col):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = rss(rows, salary_col)#-----------------------------
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column
        values = sorted(values)#-----------------------------
        values = promedio(values)#-----------------------------

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain2(true_rows, false_rows, current_uncertainty, salary_col)#--------

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [22]:
def build_tree2(rows, salary_col, poda):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split2(rows, salary_col)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0 or len(rows) < poda:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree2(true_rows, salary_col, poda)

    # Recursively build the false branch.
    false_branch = build_tree2(false_rows, salary_col, poda)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

- **Prueba 1 (Sólo variable Hits y Years)**

In [28]:
num_col_salary = 2
header = ["Hits","Years","Salary"]
poda = 30
my_tree = build_tree2(hits_years_lol, num_col_salary, poda)
print_tree(my_tree)

Is Years >= 4.5?
--> True:
  Is Hits >= 117.5?
  --> True:
    Is Years >= 7.5?
    --> True:
      Is Hits >= 147.5?
      --> True:
        Predict  {6.6: 1, 6.2: 1, 6.7: 1, 7.1: 1, 7.5: 1, 6.9: 1, 7.8: 1, 7.0: 1, 6.0: 1, 7.4: 1, 6.3: 1}
      --> False:
        Predict  {6.2: 1, 6.6: 1, 6.8: 1, 6.9: 1, 6.7: 1, 6.3: 1, 6.4: 1, 7.3: 1, 7.6: 1, 5.6: 1, 7.2: 1, 7.1: 1}
    --> False:
      Is Hits >= 185.0?
      --> True:
        Predict  {7.6: 1, 7.1: 1, 4.5: 1, 6.6: 1, 7.2: 1, 7.4: 1}
      --> False:
        Predict  {6.6: 1, 6.4: 1, 6.7: 1, 7.2: 1, 6.8: 1, 6.0: 1, 6.5: 1, 6.9: 1, 7.1: 1, 6.3: 1, 6.2: 1, 7.0: 1}
  --> False:
    Is Hits >= 76.5?
    --> True:
      Is Hits >= 94.5?
      --> True:
        Predict  {6.4: 1, 6.7: 1, 6.8: 1, 6.5: 1, 6.0: 1, 5.8: 1, 6.2: 1, 6.6: 1, 6.1: 1, 4.6: 1, 4.4: 1, 5.7: 1, 6.3: 1}
      --> False:
        Predict  {6.2: 1, 7.0: 1, 6.0: 1, 5.8: 1, 6.4: 1, 6.1: 1, 6.6: 1, 6.3: 1, 5.9: 1, 5.3: 1, 5.7: 1, 6.7: 1, 6.5: 1, 6.8: 1}
    --> False:
      

- **Prueba 2 (Todas las variables continuas)**

In [29]:
num_col_salary = 16
header = ["AtBat","Hits","HmRun","Runs","RBI","Walks","Years",    
          "CAtBat","CHits","CHmRun","CRuns","CRBI","CWalks",
          "PutOuts","Assists","Errors","Salary"]
poda = 30
my_tree = build_tree2(hitters_lol, num_col_salary, poda)
print_tree(my_tree)

Is CHits >= 419.0?
--> True:
  Is RBI >= 56.5?
  --> True:
    Is CHmRun >= 102.5?
    --> True:
      Is PutOuts >= 286.0?
      --> True:
        Predict  {6.7: 1, 6.8: 1, 6.3: 1, 7.5: 1, 7.8: 1, 6.6: 1, 7.6: 1, 6.9: 1, 7.1: 1, 7.2: 1, 7.4: 1, 7.3: 1}
      --> False:
        Predict  {6.2: 1, 7.0: 1, 6.7: 1, 6.8: 1, 7.1: 1, 7.2: 1, 6.9: 1, 7.3: 1, 6.4: 1, 6.6: 1}
    --> False:
      Is CHmRun >= 68.0?
      --> True:
        Predict  {6.2: 1, 6.7: 1, 7.6: 1, 7.1: 1, 6.9: 1, 5.6: 1, 7.0: 1, 6.0: 1, 6.6: 1, 7.2: 1}
      --> False:
        Predict  {6.2: 1, 6.4: 1, 6.5: 1, 6.6: 1, 6.7: 1, 5.9: 1, 6.3: 1, 6.8: 1, 7.4: 1}
  --> False:
    Is AtBat >= 369.0?
    --> True:
      Is RBI >= 52.5?
      --> True:
        Predict  {6.6: 1, 6.5: 1, 6.7: 1, 7.6: 1, 7.1: 1, 4.5: 1}
      --> False:
        Predict  {6.6: 1, 6.4: 1, 6.7: 1, 5.8: 1, 6.2: 1, 6.3: 1, 6.8: 1, 6.9: 1, 7.0: 1, 6.0: 1}
    --> False:
      Is PutOuts >= 157.0?
      --> True:
        Predict  {6.2: 1, 6.6: 1, 6.4: 1, 5